In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import geopandas
import dataretrieval.nwis as nwis
import contextily as cx
from matplotlib.ticker import FormatStrFormatter
import matplotlib as mpl
from pynhd import NLDI, WaterData, NHDPlusHR, NHD
import pynhd as nhd
import seaborn as sns

mpl.rcParams['pdf.fonttype'] = 42


In [8]:
## https://github.com/hyriver/pynhd


usfs = geopandas.read_file("../../data/starkey/S_USA.AdministrativeForest/S_USA.AdministrativeForest.shp")
# starkey_repro = starkey.to_crs(epsg=4269)
usfs = usfs[usfs['REGION']=='06']
walwat = usfs[usfs['FORESTNUMB']=='16']

In [9]:
walwat = walwat.reset_index(drop=True)

hr = nhd.NHD("flowline_hr")
nhdp_hr = hr.bygeom(walwat.geometry[0].bounds)
nhdp_hr = nhdp_hr.to_crs(epsg=4269)

In [5]:
## https://github.com/hyriver/pynhd

states = geopandas.read_file("../../data/mapping/cb_2018_us_state_5m/cb_2018_us_state_5m.shp")
# states = states.to_crs(epsg=4269)
st = ["Oregon"]
states = states[states["NAME"].isin(st)]
# states = unary_union(states.geometry)
# states = geopandas.GeoSeries(states)
# states = states.set_crs(epsg=4269)
states = states.reset_index(drop=True)

In [7]:
hr = nhd.NHD("flowline_hr")
nhdp_hr = hr.bygeom(states.geometry[0102000076].bounds)
nhdp_hr = nhdp_hr.to_crs(epsg=4269)

ServiceError: Service returned the following error message:
URL: https://hydro.nationalmap.gov/arcgis/rest/services/nhd/MapServer/6/query
ERROR: <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">
<TITLE>ERROR: The request could not be satisfied</TITLE>
</HEAD><BODY>
<H1>504 ERROR</H1>
<H2>The request could not be satisfied.</H2>
<HR noshade size="1px">
CloudFront attempted to establish a connection with the origin, but either the attempt failed or the origin closed the connection.
We can't connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.
<BR clear="all">
If you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.
<BR clear="all">


In [ ]:
## Just 
nhdp_hrNP = hr[(hr["fcode"]<=46007) & (hr["ftype"]==460)]
s_type = {46000: 'perennial',
             46006:'perennial',
          46003: 'non-perennial' ,
          46007: 'non-perennial'}

nhdp_hrNP['stype']=nhdp_hrNP['fcode'].map(s_type)


In [ ]:
walwatNP = geopandas.clip(nhdp_hrNP, walwat)

In [ ]:
f,ax = plt.subplots(figsize=(15, 15))

walwatNP_repro = walwatNP.to_crs(epsg=3857)
walwat_repro = walwat.to_crs(epsg=3857)

# walwatNP.plot(ax=ax,alpha = 0.5,color = walwatNP['stype'].map(color_map))
# walwat.plot(ax=ax,edgecolor="black",facecolor="none")

# plt.rcParams.update({'font.size': 18})
# ax.set_xlabel("Longitude (decDeg)")
# ax.set_ylabel("Latitude (decDeg)")
# ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
# ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))

# starkey_repro = starkey.to_crs(epsg=3857)

plt.rcParams.update({'font.size': 18})

color_map = {'perennial': 'blue',
          'non-perennial': 'green'}

walwatNP_repro.plot(ax=ax,legend=True,color = walwatNP_repro['stype'].map(color_map))


ax.arrow(-13220000, 5785000, 0, 20000, length_includes_head=True,
          head_width=5000, head_length=10000, overhang=.25, facecolor='k')
ax.text(x=-13224500, y=5775000, s='N', fontsize=30)
ax.set_xlabel("UTM Easting (m)")
ax.set_ylabel("UTM Northing (m)")
ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
plt.locator_params(axis='y', nbins=6)
plt.locator_params(axis='x', nbins=4)
cx.add_basemap(ax ,reset_extent= False,zoom=12,source=cx.providers.USGS.USTopo)
plt.savefig('../docs/wilwit_NPmap.png',dpi=300)

In [ ]:
f,ax = plt.subplots(figsize=(7, 7))
# sns.countplot(walwatNP['stype'],
#               palette=color_map)


walwatNP_count = walwatNP.groupby(['stype'])['lengthkm'].agg('sum').to_frame()
plt.rcParams.update({'font.size': 18})
colors = ['green', 'blue']
explode = (0, 0.1)
plt.pie(walwatNP_count['lengthkm'], 
        startangle=90, 
        colors=colors, 
        wedgeprops={'edgecolor': 'black'}, 
        autopct='%1.f%%', 
        explode=explode,
        labels = walwatNP_count.index)

plt.savefig('../docs/wilwit_NPpie.png',dpi=600,bbox_inches = "tight")